In [1]:
#pip install fastapi==0.114.2


In [2]:
#pip install langchain-groq

In [3]:
import os
from langchain_groq import ChatGroq


In [4]:

# Initialize and use the GooglePalm class
llm = ChatGroq(temperature=0, groq_api_key='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', model_name="llama-3.1-70b-versatile")

In [5]:
llm.invoke("Tell about me")

AIMessage(content="I don't have any information about you. Our conversation just started, and I don't have any prior knowledge about you. I'm a large language model, I don't have the ability to retain information about individual users or access any personal data.\n\nHowever, I can try to get to know you better through our conversation. If you'd like to share something about yourself, I'm here to listen and respond. What would you like to talk about? Your interests, hobbies, or something else?", response_metadata={'token_usage': {'completion_tokens': 101, 'prompt_tokens': 38, 'total_tokens': 139, 'completion_time': 0.404, 'prompt_time': 0.009089689, 'queue_time': 0.00488622, 'total_time': 0.413089689}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_5c5d1b5cfb', 'finish_reason': 'stop', 'logprobs': None}, id='run-356235e4-e9ce-4bb1-a42d-b9616dce17a4-0', usage_metadata={'input_tokens': 38, 'output_tokens': 101, 'total_tokens': 139})

In [6]:
from langchain_community.utilities import SQLDatabase



In [7]:
db_user = "root"
db_password = "123456"
db_host = "localhost"
db_name = "estore_tshirts"

db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",sample_rows_in_table_info=3)

print(db.table_info)


CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))
)COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB DEFAULT CHARSET=utf8mb4

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount
1	1	10.00
2	2	15.00
3	3	20.00
*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB DEFAULT CHARSET=utf8mb4

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock

In [8]:
from langchain.chains import create_sql_query_chain

chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "How many t-shirts do we have left for nike in extra small size and red color?"})
response


"Question: How many t-shirts do we have left for nike in extra small size and red color?\nSQLQuery: SELECT `stock_quantity` FROM t_shirts WHERE `brand` = 'Nike' AND `size` = 'XS' AND `color` = 'Red' LIMIT 1;"

In [9]:
def find_sql_query(response):
    # Clean the response by replacing newlines with spaces
    cleaned_response = response.replace("\n", " ").replace("```", "").strip()
    
    # Find the starting point of the SQL query using 'SELECT'
    select_start = cleaned_response.find("SELECT")
    
    if select_start == -1:
        print("Error: Could not find 'SELECT' in the response")
        return None
    
    # Extract the SQL query starting from 'SELECT' to the end of the string
    sql_query = cleaned_response[select_start:].strip()
    
    # Print and return the extracted SQL query
    print("Extracted SQL Query:", sql_query)
    return sql_query
   
    
    # Print and return the extracted SQL query
    print("Extracted SQL Query:", sql_query)
    return sql_query


In [10]:
sql_query = find_sql_query(response)

Extracted SQL Query: SELECT `stock_quantity` FROM t_shirts WHERE `brand` = 'Nike' AND `size` = 'XS' AND `color` = 'Red' LIMIT 1;


In [11]:
# Execute the query using the SQLDatabase object
result = db.run(sql_query)

# Print the result
print("Query Result:", result)

Query Result: [(11,)]


In [12]:
response2 = chain.invoke({"question":"How much is the price of the inventory for all small size t-shirts?"})
response2

"Question: How much is the price of the inventory for all small size t-shirts?\nSQLQuery: \nSELECT SUM(`price` * `stock_quantity`) AS `total_price` \nFROM `t_shirts` \nWHERE `size` = 'S';"

In [13]:
sql_query = find_sql_query(response2)

Extracted SQL Query: SELECT SUM(`price` * `stock_quantity`) AS `total_price`  FROM `t_shirts`  WHERE `size` = 'S';


In [14]:
result2 = db.run(sql_query)
result2

"[(Decimal('13136'),)]"

In [15]:
response3 = chain.invoke({"question":"If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?"})
response3

"Question: If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?\n\nSQLQuery: \n```sql\nSELECT \n    SUM((ts.`price` * (1 - (d.`pct_discount` / 100))) * ts.`stock_quantity`) AS total_revenue\nFROM \n    t_shirts ts\nJOIN \n    discounts d ON ts.`t_shirt_id` = d.`t_shirt_id`\nWHERE \n    ts.`brand` = 'Levi';\n```"

In [16]:
sql_query = find_sql_query(response3)

Extracted SQL Query: SELECT      SUM((ts.`price` * (1 - (d.`pct_discount` / 100))) * ts.`stock_quantity`) AS total_revenue FROM      t_shirts ts JOIN      discounts d ON ts.`t_shirt_id` = d.`t_shirt_id` WHERE      ts.`brand` = 'Levi';


In [17]:
result3 = db.run(sql_query)
result3

"[(Decimal('3547.100000'),)]"

In [18]:
response4 = chain.invoke({"question":"If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?"})
response4

"Question: If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?\nSQLQuery: \nSELECT SUM(`price` * `stock_quantity`) AS total_revenue \nFROM t_shirts \nWHERE `brand` = 'Levi';"

In [19]:
sql_query = find_sql_query(response2)
result4 = db.run(sql_query)
result4

Extracted SQL Query: SELECT SUM(`price` * `stock_quantity`) AS `total_price`  FROM `t_shirts`  WHERE `size` = 'S';


"[(Decimal('13136'),)]"

In [20]:
response5 = chain.invoke({"question":"How many white color Levi's t shirts we have available?"})
response5

"Question: How many white color Levi's t shirts we have available?\nSQLQuery: SELECT `stock_quantity` FROM t_shirts WHERE `brand` = 'Levi' AND `color` = 'White'"

In [44]:
sql_query = find_sql_query(response5)
result5 = db.run(sql_query)
result5

Extracted SQL Query: SELECT `stock_quantity` FROM t_shirts WHERE `brand` = 'Levi' AND `color` = 'White'


'[(95,), (21,), (62,), (45,)]'

In [22]:
def convert_number(result):
    # Remove the outer brackets and parentheses, and extract the decimal part
    cleaned_result = result[result.find("('") + 2:result.find("')")]
    number_string = cleaned_result.strip("()").split(",")[0]
    return number_string

In [23]:
res= db.run("SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'Red' AND size = 'XS'")
ans1 = convert_number(res)
ans1


'11'

In [24]:
res= db.run("SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'")
ans2 = convert_number(res)
ans2

'13136'

In [25]:
res= db.run("SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from (select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi' group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id")
ans3 = convert_number(res)
ans3

'22703.100000'

In [26]:
res= db.run("SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'")
ans4 = convert_number(res)
ans4

'24042'

In [27]:
res= db.run("SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'")
ans5 = convert_number(res)
ans5

'223'

In [28]:
examples = [
    {'Question' : "How many t-shirts do we have left for Nike in XS size and red color?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'Red' AND size = 'XS'",
     'SQLResult': "Result of the SQL query",
     'Answer' : ans1},
    {'Question': "How much is the total price of the inventory for all S-size t-shirts?",
     'SQLQuery':"SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'",
     'SQLResult': "Result of the SQL query",
     'Answer': ans2},
    {'Question': "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?" ,
     'SQLQuery' : """SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """,
     'SQLResult': "Result of the SQL query",
     'Answer': ans3} ,
     {'Question' : "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?" ,
      'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
      'SQLResult': "Result of the SQL query",
      'Answer' : ans4},
    {'Question': "How many white color Levi's shirt I have?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
     'SQLResult': "Result of the SQL query",
     'Answer' : ans5
     }
]

In [29]:
pip install sentence-transformers


Note: you may need to restart the kernel to use updated packages.


In [30]:
pip install tf-keras


Note: you may need to restart the kernel to use updated packages.


In [31]:
from langchain.prompts import SemanticSimilarityExampleSelector
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma


embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

to_vectorize = [" ".join(example.values()) for example in examples]

C:\Users\anwit\anaconda3\Lib\site-packages\pydantic\_internal\_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
C:\Users\anwit\AppData\Local\Temp\ipykernel_4512\2625274983.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
C:\Users\anwit\anaconda3\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tq

C:\Users\anwit\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [32]:
to_vectorize

["How many t-shirts do we have left for Nike in XS size and red color? SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'Red' AND size = 'XS' Result of the SQL query 11",
 "How much is the total price of the inventory for all S-size t-shirts? SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S' Result of the SQL query 13136",
 "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)? SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from\n(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'\ngroup by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id\n  Result of the SQL query 22703.100000",
 "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount? SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'

In [33]:
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=examples)

In [34]:
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

example_selector.select_examples({"Question": "How many Adidas T shirts I have left in my store?"})

[{'Answer': '11',
  'Question': 'How many t-shirts do we have left for Nike in XS size and red color?',
  'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'Red' AND size = 'XS'",
  'SQLResult': 'Result of the SQL query'},
 {'Answer': '24042',
  'Question': 'If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?',
  'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
  'SQLResult': 'Result of the SQL query'}]

In [37]:
from langchain.prompts import FewShotPromptTemplate
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt

print(_mysql_prompt)

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of

In [38]:
from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult","Answer",],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",
)

In [39]:
few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=mysql_prompt,
    suffix=PROMPT_SUFFIX,
    input_variables=["input", "table_info", "top_k"], #These variables are used in the prefix and suffix
)

In [42]:
#new_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, prompt=few_shot_prompt)
chain = create_sql_query_chain(llm, db, prompt=few_shot_prompt)
response = chain.invoke({"question": "How many white color Levi's shirt I have?"})
response

"Question: How many white color Levi's shirt I have?\nSQLQuery: SELECT sum(`stock_quantity`) FROM `t_shirts` WHERE `brand` = 'Levi' AND `color` = 'White'"

In [43]:
sql_query = find_sql_query(response)
result = db.run(sql_query)
result

Extracted SQL Query: SELECT sum(`stock_quantity`) FROM `t_shirts` WHERE `brand` = 'Levi' AND `color` = 'White'


"[(Decimal('223'),)]"